In [1]:
import pylab as pl
import numpy as np
import pprint as pp
import pandas as pd
import random
from datetime import datetime
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
class DecisionTree:
    """ A basic Decision Tree"""

    def __init__(self):
        """ Constructor """

    def read_data(self, filename, limit=None):
        fid = open(filename, "r")
        data = []
        d = []
        
        for l,line in enumerate(fid.readlines()):
            if limit is not None:
                if l == limit+1:
                    break
            d.append(line.strip())
            
        for d1 in d:
            data.append(d1.split(","))
        fid.close()

        self.featureNames = data[0]
        self.featureNames = self.featureNames[:-1]
        data = data[1:]
        self.classes = []
        for d in range(len(data)):
            self.classes.append(data[d][-1])
            data[d] = data[d][:-1]

        return data, self.classes, self.featureNames

    def classify(self, tree, datapoint):

        if type(tree) == type("string"):
            # Have reached a leaf
            return tree
        else:
            a = list(tree.keys())[0]
            for i in range(len(self.featureNames)):
                if self.featureNames[i] == a:
                    break

            try:
                t = tree[a][datapoint[i]]
                return self.classify(t, datapoint)
            except:
                return None

    def classifyAll(self, tree, data):
        results = []
        for i in range(len(data)):
            results.append(self.classify(tree, data[i]))
        return results

    def make_tree(self, data, classes, featureNames, maxlevel=-1, level=0, forest=0):
        """ The main function, which recursively constructs the tree"""

        nData = len(data)
        nFeatures = len(data[0])

        try:
            self.featureNames
        except:
            self.featureNames = featureNames

        # List the possible classes
        newClasses = []
        for aclass in classes:
            if newClasses.count(aclass) == 0:
                newClasses.append(aclass)

        # Compute the default class (and total entropy)
        frequency = np.zeros(len(newClasses))

        totalEntropy = 0
        totalGini = 0
        index = 0
        for aclass in newClasses:
            frequency[index] = classes.count(aclass)
            totalEntropy += self.calc_entropy(float(frequency[index])/nData)
            totalGini += (float(frequency[index])/nData)**2

            index += 1

        totalGini = 1 - totalGini
        default = classes[np.argmax(frequency)]

        if nData == 0 or nFeatures == 0 or (maxlevel >= 0 and level > maxlevel):
            # Have reached an empty branch
            return default
        elif classes.count(classes[0]) == nData:
            # Only 1 class remains
            return classes[0]
        else:

            # Choose which feature is best
            gain = np.zeros(nFeatures)
            ggain = np.zeros(nFeatures)
            featureSet = list(range(nFeatures))
            if forest != 0:
                np.random.shuffle(featureSet)
                featureSet = featureSet[0:forest]
                
            for feature in featureSet:
                g, gg = self.calc_info_gain(data, classes, feature)
                gain[feature] = totalEntropy - g
                ggain[feature] = totalGini - gg

            bestFeature = np.argmax(gain)
            tree = {featureNames[bestFeature]: {}}

            # List the values that bestFeature can take
            values = []
            for datapoint in data:
                if datapoint[feature] not in values:
                    values.append(datapoint[bestFeature])

            for value in values:
                # Find the datapoints with each feature value
                newData = []
                newClasses = []
                index = 0
                for datapoint in data:
                    if datapoint[bestFeature] == value:
                        if bestFeature == 0:
                            newdatapoint = datapoint[1:]
                            newNames = featureNames[1:]
                        elif bestFeature == nFeatures:
                            newdatapoint = datapoint[:-1]
                            newNames = featureNames[:-1]
                        else:
                            newdatapoint = datapoint[:bestFeature]
                            newdatapoint.extend(datapoint[bestFeature+1:])
                            newNames = featureNames[:bestFeature]
                            newNames.extend(featureNames[bestFeature+1:])
                        newData.append(newdatapoint)
                        newClasses.append(classes[index])
                    index += 1

                # Now recurse to the next level
                subtree = self.make_tree(
                    newData, newClasses, newNames, maxlevel, level+1, forest)

                # And on returning, add the subtree on to the tree
                tree[featureNames[bestFeature]][value] = subtree

            return tree

    def printTree(self, tree, name):
        if type(tree) == dict:
            print(name, tree.keys()[0])
            for item in tree.values()[0].keys():
                print (name, item)
                self.printTree(tree.values()[0][item], name + "\t")
        else:
            print (name, "\t->\t", tree)

    def calc_entropy(self, p):
        if p != 0:
            return -p * np.log2(p)
        else:
            return 0

    def calc_info_gain(self, data, classes, feature):

        # Calculates the information gain based on both entropy and the Gini impurity
        gain = 0
        ggain = 0
        nData = len(data)

        # List the values that feature can take

        values = []
        for datapoint in data:
            if datapoint[feature] not in values:
                values.append(datapoint[feature])

        featureCounts = np.zeros(len(values))
        entropy = np.zeros(len(values))
        gini = np.zeros(len(values))
        valueIndex = 0
        # Find where those values appear in data[feature] and the corresponding class
        for value in values:
            dataIndex = 0
            newClasses = []
            for datapoint in data:
                if datapoint[feature] == value:
                    featureCounts[valueIndex] += 1
                    newClasses.append(classes[dataIndex])
                dataIndex += 1

            # Get the values in newClasses
            classValues = []
            for aclass in newClasses:
                if classValues.count(aclass) == 0:
                    classValues.append(aclass)

            classCounts = np.zeros(len(classValues))
            classIndex = 0
            for classValue in classValues:
                for aclass in newClasses:
                    if aclass == classValue:
                        classCounts[classIndex] += 1
                classIndex += 1

            for classIndex in range(len(classValues)):
                entropy[valueIndex] += self.calc_entropy(
                    float(classCounts[classIndex])/np.sum(classCounts))
                gini[valueIndex] += (float(classCounts[classIndex]
                                           )/np.sum(classCounts))**2

            # Computes both the Gini gain and the entropy
            gain = gain + \
                float(featureCounts[valueIndex])/nData * entropy[valueIndex]
            ggain = ggain + \
                float(featureCounts[valueIndex])/nData * gini[valueIndex]
            valueIndex += 1
        return gain, 1-ggain

In [5]:

def split_set(X,C, rate):
    training_set_size = int(len(X)*rate)
    test_set_size     = len(X) - training_set_size
    
    ids = []
    
    for i in range(test_set_size):
        v = int(random.random()*len(X))
        while( v in ids):
            v = int(random.random()*len(X))
        ids.append(v)
    
    train_set = []
    train_set_answers = []
    test_set  = []
    test_set_answers = []
    
    for x in range(len(X)):
        if x in ids:
            test_set.append(X[x])
            test_set_answers.append(C[x])
        else:
            train_set.append(X[x])
            train_set_answers.append(C[x])
    
    return [train_set,train_set_answers, test_set, test_set_answers]

random_forest = []

def read_data(filename, limit=None):
    fid = open(filename, "r")
    data = []
    d = []
        
    for l,line in enumerate(fid.readlines()):
        if limit is not None:
            if l == limit+1:
                break
        d.append(line.strip())
            
    for d1 in d:
        data.append(d1.split(","))
    fid.close()

    featureNames = data[0]
    featureNames = featureNames[:-1]
    data = data[1:]
    classes = []
    for d in range(len(data)):
        classes.append(data[d][-1])
        data[d] = data[d][:-1]

    return data, classes, featureNames

forestsize = 10

X = read_data("car.data", 1000)

train_set = split_set(X[0],X[1], 0.75)
test_set     = train_set[2]
test_set_ans = train_set[3]
train_set_ans= train_set[1]
train_set    = train_set[0]

v = []

#print(test_set)
#print(train_set)

T = DecisionTree()

print( "Building trees ... ( Takes a lot of time )")

tim = datetime.now()

for i in range(forestsize):
    tim3 = datetime.now()
    
    v = split_set(train_set, train_set_ans ,0.5)
    
    random_forest.append( DecisionTree() )
    
    random_forest[i] = T.make_tree(
        v[0],
        v[1], 
        X[2],
        len(X[2]),
        forest = len(X[2])
    )
    
    tim2 = datetime.now()

    print(i + 1, " of ",forestsize ," Tree : ready after ", (tim2-tim3).seconds, " seconds" )
    
output = {}
output_one ={}

correct     = [0,0]
correct_one = [0,0]

flag = True

for j in range(len(v[2])):
    decision = T.classify(random_forest[0],v[2][j])

    if decision is not None :
                        
        if not decision in output_one.keys():
            output_one[decision] = 1
        else:
            output_one[decision] = output_one[decision] + 1
        if decision == v[3][j]:
            correct_one[0] += 1
        else :
            correct_one[1] += 1

    else:
        if not "None" in output_one.keys():
            output_one["None"] = 1
        else:
            output_one["None"] = output_one["None"] + 1


p = []

for i in range(len(v[2])):
    p.append({})
            
for j in range(len(v[2])):
    for tree in random_forest:
        decision = T.classify(tree,v[2][j])
        if decision is None:
            if not "None" in p[j].keys():
                p[j]["None"] = 1
            else:
                p[j]["None"] += 1
        else:
            if not decision in p[j].keys():
                p[j][decision] = 1
            else:
                p[j][decision] += 1

#print(p)
                
for j in range(len(v[2])):
    best  = -1
    value = ""
    
    for k in p[j]:
        if p[j][k] > best:
            value = k
            best  = p[j][k] 
            
    if not value in output.keys():
        output[value] = 1
    else:
        output[value] =  output[value] + 1
        
    if value == v[3][j]:
        correct[0] += 1
    else :
        correct[1] += 1

        
print(" ... Done after ", (tim2-tim).seconds, " seconds" )
        
print(output_one)
#print(correct_one)       
        
print(output)
#print(correct)


    
    

Building trees ... ( Takes a lot of time )
1  of  10  Tree : ready after  1003  seconds
2  of  10  Tree : ready after  329  seconds
3  of  10  Tree : ready after  13  seconds
4  of  10  Tree : ready after  545  seconds
5  of  10  Tree : ready after  672  seconds
6  of  10  Tree : ready after  7  seconds
7  of  10  Tree : ready after  9  seconds
8  of  10  Tree : ready after  687  seconds
9  of  10  Tree : ready after  317  seconds
10  of  10  Tree : ready after  524  seconds
 ... Done after  4111  seconds
{'unacc': 293, 'None': 16, 'acc': 66}
{'unacc': 292, 'acc': 76, 'None': 7}


In [6]:
print("One Tree")
print("   Correctly predicted : ", correct_one[0], " out of ", len(v[2]))
print("   Accuracy : ", correct_one[0]/len(v[2]), "%")

print("10 Trees")
print("   Correctly predicted : ", correct[0], " out of ", len(v[2]))
print("   Accuracy : ", correct[0]/len(v[2]), "%")

One Tree
   Correctly predicted :  354  out of  375
   Accuracy :  0.944 %
10 Trees
   Correctly predicted :  367  out of  375
   Accuracy :  0.9786666666666667 %
